# Nuclei data exploration

In [1]:
import pandas as pd
import numpy as np
import zipfile
import re
from PIL import Image
from io import BytesIO
from collections import Counter

## Init

In [16]:
train_zip = zipfile.ZipFile('./stage1_train.zip')

In [53]:
def load_png(png, zipped_pngs):
    png_data = zipped_pngs.read(png)
    return Image.open(BytesIO(png_data))


train_images = [(x[:64], load_png(x, train_zip)) for x in train_zip.namelist() if re.search(r'images/.', x)]
train_masks  = [(x[:64], load_png(x, train_zip)) for x in train_zip.namelist() if re.search(r'masks/.', x)]
train_images = pd.DataFrame(train_images, columns = ['hash', 'image'])
train_masks  = pd.DataFrame(train_masks, columns = ['hash', 'mask'])
train        = pd.merge(train_images, train_masks, 'outer', 'hash')

## Images features

In [55]:
train['length'] = [x.size for x in train['image']]

In [56]:
train

,hash,image,mask,length
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
1,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
2,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
3,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
4,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
5,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
6,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
7,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
8,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"
9,00071198d059ba7f5914a526d124d28e6d010c92466da2...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=L ...,"(256, 256)"


## Thoughts

Try a generalization test by isolating a class of images and predict on it